# Week 2

The goal of this second week is to get a first glance at the Courant Snyder parameters

These parameters are dependant on the length $L$ of the FODO cell and the gradient of magnetic field $\frac{\partial B_{y}}{\partial x}$

When we take the limit of quadrupoles being thin lenses (thus thin $L$), we get the following equations

$$\begin{array}{c}
L \rightarrow 0 \\
k_{1} L \rightarrow \frac{1}{f} \\
k_{1}=\frac{q}{P_{0}} \frac{\partial B_{y}}{\partial x}
\end{array}$$

$P_0 = \beta_0\gamma_0 mc$ is the initial canonical momentum. Moreover, $\beta\gamma = \sqrt{\gamma^2-1}$ and $\gamma = \frac{E}{mc^2}$.
$q$ is the charge of the particle we consider.


In [2]:
import numpy as np
import math as m
import pandas

In [4]:
E_e = 45 ; # energy of the positron in GeV
E_mu = 22 ; # energy of the muon in GeV

q_e = 1; #charge of a positron in units of an electron charge
q_mu = -1; #charge of a muon in units of an electron charge

m_e = 0.511*10**(-3); # mass of the positron in GeV/c^2
m_mu = 0.10565; # mass of the muon in GeV/c^2


gamma_e = E_e/(m_e) # c = 1
gamma_mu = E_mu/(m_mu) # c = 1

L = 0.1; # length in meters TO BE SET
f = np.sqrt(2); # focal length
k_1 = 1/(f*L);
P_e = m_e*np.sqrt(gamma_e**2 - 1) # canonical momentum of the positron in GeV/c

dB = P_e/q_e*k_1; # gradient of magnetic field




The transfer matrix for a quadrupole is the following
$$R_{\mathrm{F}}=\left(\begin{array}{rrrrr}
1 & 0 & 0 & 0 & 0 \\
-\frac{1}{f} & 1 & 0 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 & 0 \\
0 & 0 & \frac{1}{f} & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 & 0 & 1
\end{array}\right)$$

The transfer matrix for a drift portion is the following

$$R_{\mathrm{drift}}=\left(\begin{array}{cccccc}
1 & L & 0 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 & 0 & 0 \\
0 & 0 & 1 & L & 0 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 1 & \frac{L}{\beta_{0}^{2} \gamma_{0}^{2}} \\
0 & 0 & 0 & 0 & 0 & 1
\end{array}\right)$$

For an entire FODO cell, with a starting point $s=s_0$ (entrance of the focusing quadrupole), we get the matrix $R = R_{drift}R_D R_{drift} R_F = R=\left(\begin{array}{ccc}
R_{x} & 0 & 0 \\
0 & R_{y} & 0 \\
0 & 0 & R_{z}
\end{array}\right)$

Where 
$$ R_{x}=\left(\begin{array}{cc}
1-\frac{L}{f}\left(1+\frac{L}{f}\right) & \frac{L}{f}(2 f+L) \\
-\frac{L}{f^{2}} & 1+\frac{L}{f}
\end{array}\right) \qquad R_{y}=\left(\begin{array}{cc}
1+\frac{L}{f}\left(1-\frac{L}{f}\right) & \frac{L}{f}(2 f-L) \\
-\frac{L}{f^{2}} & 1-\frac{L}{f} \end{array}\right)  \qquad R_{z}=\left(\begin{array}{ll}
1 & \frac{2 L}{\beta_{0}^{2} \gamma_{0}^{2}} \\
0 & 1 \end{array}\right)
$$



In [5]:
# Building the transfer matrix of the FODO cell
Rx1 = 1 - L/f*(1+f); 
Rx2 = L/f*(2*f+L);
Rx3 = -L/(f**2);
Rx4 = 1+L/f;

Ry1 = 1 + L/f*(1-f); 
Ry2 = L/f*(2*f-L);
Ry3 = -L/f**2;
Ry4 = 1-L/f;

Rx = np.matrix([[Rx1, Rx2],
      [Rx3, Rx4]]);
Ry = np.matrix([[Ry1, Ry2],
      [Ry3, Ry4]]);
Rz = np.matrix([[1, 2*L/(gamma_e**1 -1)],
      [0, 1]]); # because gamma*beta = sqrt(gamma^2 - 1)



Now we want the coefficients. We will for that use the reference book *Beam Dynamics in High Energy Particle Accelerators* to get the appropriate formulas.

We know that in our case (a particle propagating in the x direction as its longitudinal direction), $ Tr(R_x) = 2 cos(\mu_x) $ where $\mu_x$ is the betatron phase advance. The trace is an invariant under changing the starting point, so does the phase advance.



We have the formula 

$$R_{x}=\left(\begin{array}{cc}
\cos \mu_{x}+\alpha_{x} \sin \mu_{x} & \beta_{x} \sin \mu_{x} \\
-\gamma_{x} \sin \mu_{x} & \cos \mu_{x}-\alpha_{x} \sin \mu_{x}
\end{array}\right)$$

Which allows us to find the values of the Courant Snyder parameters, using only the values found for $R_x$ and the value calculated for $\mu_x$.

In [8]:
# mu_x = m.acos( Rx.trace());

tr= np.asscalar(Rx.trace());
mu_x = m.acos(tr/2) 
alpha_x0 = (Rx1 -np.cos(mu_x))/np.sin(mu_x)
beta_x0 = Rx2/np.sin(mu_x)
gamma_x0 = -Rx3/np.sin(mu_x)


<ipython-input-8-d9cae43aba6f>:3: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  tr= np.asscalar(Rx.trace());


$$R(s_1,s_0) = \left(\begin{array}{cc}
\sqrt{\frac{\beta_{x 1}}{\beta_{x 0}}}\left(\cos \mu_{x}+\alpha_{x 0} \sin \mu_{x}\right) & \sqrt{\beta_{x 0} \beta_{x 1}} \sin \mu_{x} \\
\frac{\alpha_{x 0}-\alpha_{x 1}}{\sqrt{\beta_{x 0} \beta_{x 1}}} \cos \mu_{x}-\frac{1+\alpha_{x 0} \alpha_{x 1}}{\sqrt{\beta_{x 0} \beta_{x 1}}} \sin \mu_{x} & \sqrt{\frac{\beta_{x 0}}{\beta_{x 1}}}\left(\cos \mu_{x}-\alpha_{x 1} \sin \mu_{x}\right)
\end{array}\right)$$



To get the evolution of the parameters in the FODO lattice, we have to get the evolution of the parameters in the drift space, we can write the transfer matrix over a distance $s$ (4.90):

$$R_{x}=\left(\begin{array}{ll}
1 & s \\
0 & 1
\end{array}\right)$$

Then, performing the matrix multiplication in (4.29), we find:

$$\begin{array}{l}
\beta_{x}(s)=\beta_{x 0}+\frac{s^{2}}{\beta_{x 0}} \\
\alpha_{x}(s)=-\frac{s}{\beta_{x 0}}
\end{array}$$

Moreover we have the equation (4.74) which gives us $\beta_x \gamma_x - \alpha_x^2 = -1$, this gives 



In [ ]:
# we are going to devide the 

The action variable is : 
$$J_{x}=\frac{1}{2}\left(\begin{array}{l} x  & p_{x} \end{array}\right) A_{x}\left(\begin{array}{c}
x \\
p_{x}
\end{array}\right)
$$

also written $J_x = \frac{1}{2}(\gamma_x x^2 + 2 \alpha_x x p_x + \beta_x p_x^2)$ and should be invariant along the beam line (to be verified)

The angle variable is : $\tan \phi_{x}=-\beta_{x} \frac{p_{x}}{x}-\alpha_{x}$

The coordinates along the $x$ axis can be expressed as 
$$\begin{aligned}
x &=\sqrt{2 \beta_{x} J_{x}} \cos \phi_{x} \\
p_{x} &=-\sqrt{\frac{2 J_{x}}{\beta_{x}}}\left(\sin \phi_{x}+\alpha_{x} \cos \phi_{x}\right)
\end{aligned}$$ (eq 4.36 and 4.37 of the book)


The area of the ellipse formed by these coordinates should be $2 \pi J_x$


Questions: 

* I have to find what the book means by $R_{Fx}$ (it says "2 x 2 sub-matrix (the first block diagonal) of the transfer matrix for the horizontally focusing quadrupole"). and the first expression of $R(s_1,s_0)$
